In [1]:
import pandas as pd
import warnings

from model_tool.fundamental._fundamental_preproc import (
    FundamentalPreproc,
    Fundamental2PivotTb,
    PivotTbController,
)
from model_tool.fundamental._ohlcv_preproc import OhlcvPreproc, OhlcvRoller
from model_tool.fundamental._model import ReprtDfCoefModel

warnings.filterwarnings(action="ignore")

In [2]:
fundamental_raw_df = pd.read_csv(
    "./data/dart/ofs_fundamental_df.csv", index_col=0
)

In [3]:
fundamental_preproc = FundamentalPreproc(fundamental_raw_df)
fundamental_df = fundamental_preproc()

In [4]:
fundamental2pivot_tb = Fundamental2PivotTb(fundamental_df)
pivot_tb = fundamental2pivot_tb()

In [5]:
pivot_tb_controller = PivotTbController(pivot_tb)
latest_n_reprt_df = pivot_tb_controller.get_latest_n_reprt("11011", 3)

In [6]:
reprt_df_coef_model = ReprtDfCoefModel(latest_n_reprt_df)
reprt_coef_df = reprt_df_coef_model.get_coef_df(
    columns=["당기순이익", "영업이익", "유동자산"]
)

In [7]:
reprt_coef_df

,당기순이익,영업이익,유동자산
stock_code,,,
000020,5.715933e+09,-8.499164e+08,-1.489449e+10
000040,-4.111994e+09,-2.649943e+09,5.570503e+08
000050,-1.362227e+10,3.562091e+09,4.662656e+09
000070,-1.904502e+10,-9.032143e+10,4.088643e+10
000080,-2.560730e+10,-2.433644e+10,-1.577037e+11
...,...,...,...
950160,1.409860e+07,1.263844e+07,-5.904424e+06
950170,3.968722e+09,3.619156e+09,-1.461501e+09
950190,-8.940570e+07,-1.083197e+08,-3.171536e+08


In [8]:
latest_reprt_df = pivot_tb_controller.get_latest_reprt("11011")

In [9]:
kosdaq_ohlcv_df = pd.read_csv("./data/fdr/kosdaq_ohlcv_df.csv", index_col=0)
kospi_ohlcv_df = pd.read_csv("./data/fdr/kospi_ohlcv_df.csv", index_col=0)

stock_info_df = pd.read_csv("./data/fdr/stock_info.csv", index_col=0)

In [10]:
ohlcv_preproc = OhlcvPreproc(kospi_ohlcv_df, kosdaq_ohlcv_df)
ohlcv_df = ohlcv_preproc()

In [11]:
ohlcv_roller = OhlcvRoller(ohlcv_df)
front_rolled_price = ohlcv_roller.front_rolling_mean(30)

In [12]:
code_share_dict = stock_info_df.set_index("stock_code")[
    "stock_share"
].to_dict()

In [13]:
fundamental_price_df = pd.merge_asof(
    left=latest_reprt_df.sort_values("reprt_date"),
    right=front_rolled_price.sort_values("date"),
    by="stock_code",
    left_on="reprt_date",
    right_on="date",
    direction="nearest",
)
fundamental_price_df["shares"] = fundamental_price_df["stock_code"].map(
    code_share_dict
)
fp_df = fundamental_price_df.set_index("stock_code")

In [14]:
fp_df["자본총계"] = fp_df["자산총계"] - fp_df["부채총계"]
fp_df["비유동자본"] = fp_df["비유동자산"] - fp_df["비유동부채"]
fp_df["유동자본"] = fp_df["유동자산"] - fp_df["유동부채"]

In [16]:
net_per_series = (fp_df["price"] / (fp_df["당기순이익"] / fp_df["shares"])).rename('net_per')
operation_per_series = (fp_df["price"] / (fp_df["영업이익"] / fp_df["shares"])).rename("operation_per")
raw_pbr_series = (fp_df["price"] / (fp_df["자본총계"] / fp_df["shares"])).rename("raw_pbr")
net_pbr_series = (fp_df["price"] / (fp_df["유동자본"] / fp_df["shares"])).rename("net_pbr")

In [17]:
fp_main_df=pd.concat(
    [net_per_series, operation_per_series, raw_pbr_series, net_pbr_series],
    axis=1
)

In [18]:
stock_code_a = reprt_coef_df[
    (reprt_coef_df["당기순이익"] > 0)
    & (reprt_coef_df["영업이익"] > 0)
    & (reprt_coef_df["유동자산"] > 0)
].index

In [20]:
stock_code_b = fp_main_df[
    (fp_main_df["net_per"] > 0)
    & (fp_main_df["operation_per"] > 0)
    & (fp_main_df["raw_pbr"] > 0)
    & (fp_main_df["net_pbr"] > 0)
    & (2 > fp_main_df["net_pbr"])
].index

In [21]:
len(set(stock_code_a) & set(stock_code_b))

103

In [23]:
fundamental_result=stock_info_df[stock_info_df['stock_code'].isin(set(stock_code_a) & set(stock_code_b))]

In [24]:
fundamental_result.to_csv("./fundamental_result.csv")